## Imports

In [1]:
# DO NOT CHANGE THESE LINES.
using Suppressor
@suppress begin
    using DataFrames
    using LazyJSON
    using GLM 
    using MLJ 
    using MLJBase
    using CSV
    using Serialization
    using MLJScientificTypes
    using CategoricalArrays
    using JLD2
end


## Paths

In [2]:
# DO NOT CHANGE THESE LINES 
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
OUTPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "outputs")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.jld2")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
PREDICTIONS_DIR = joinpath(OUTPUT_DIR, "predictions")
PREDICTIONS_FILE = joinpath(PREDICTIONS_DIR, "predictions.csv")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")
TARGET_MAPPING = joinpath(MODEL_ARTIFACTS_PATH, "target_mapping.ser")


if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema

In [3]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)
features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

if length(target_classes) == 2
    negative_class = target_classes[1]
    positive_class = target_classes[2]
end



"1"

### Reading test data.

In [4]:
file_name = filter(x -> occursin(".csv", x), readdir(TEST_DIR))[1]
file_path = joinpath(TEST_DIR, file_name)
df = DataFrame(CSV.File(file_path))

,id,color,number
,String7,String7?,Float64?
1,DQYUN3,Red,75.0
2,QIP98Q,Red,87.0
3,OAEDOE,Green,56.0
4,ZFV8YU,Red,46.0
5,L9EKA0,Blue,59.0
6,9VW3RT,Green,68.0
7,MQC1A7,Red,31.0
8,5SOWOM,Green,63.0
9,76RKQ4,Blue,33.0


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [5]:
imputation_values = open(deserialize, IMPUTATION_FILE)
for column in nullable_features
    df[!, Symbol(column)] .= coalesce.(df[!, Symbol(column)], get(imputation_values, string(column), missing))
end

# Saving the id column in a different variable
ids = df[!, Symbol(id_feature)]

# Dropping the id and target from the DataFrame
select!(df, Not([Symbol(id_feature)]))

,color,number
,String7,Float64
1,Red,75.0
2,Red,87.0
3,Green,56.0
4,Red,46.0
5,Blue,59.0
6,Green,68.0
7,Red,31.0
8,Green,63.0
9,Blue,33.0


### Encoding
We encode the data using the same encoder that we saved during training.

In [6]:
loaded_top_categories = open(deserialize, TOP_CATEGORIES)

# Function to one-hot encode only the top 10 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end



one_hot_top_categories!(df, loaded_top_categories)

### Making predictions
Using the model saved during training.

In [7]:
# Load the saved model
loaded_mapping = open(deserialize, TARGET_MAPPING)
model = open(deserialize, PREDICTOR_FILE_PATH)

# Make predictions
log_odds_predictions = GLM.predict(model, Matrix(df))
probabilities = 1 ./ (1 .+ exp.(-log_odds_predictions))
threshold = 0.5
integer_predictions = ifelse.(probabilities .> threshold, 1, 0)
label_predictions = [loaded_mapping[pred] for pred in integer_predictions]
target_levels = open(deserialize, TARGET_LEVELS)
label_predictions = target_levels[integer_predictions .+ 1]


40-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

### Creating predictions DataFrame.

In [8]:
result_df = DataFrame()

result_df[!, Symbol(negative_class)] = 1 .- probabilities
result_df[!, Symbol(positive_class)] = probabilities
result_df[!, id_feature] = ids
CSV.write(PREDICTIONS_FILE, result_df)
result_df

,0,1,id
,Float64,Float64,String7
1,0.279483,0.720517,DQYUN3
2,0.272001,0.727999,QIP98Q
3,0.5,0.5,OAEDOE
4,0.388088,0.611912,ZFV8YU
5,0.496986,0.503014,L9EKA0
6,0.5,0.5,9VW3RT
7,0.463466,0.536534,MQC1A7
8,0.5,0.5,5SOWOM
9,0.499804,0.500196,76RKQ4
